In [ ]:
import tensorflow as tf
import plotly as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [ ]:
df = pd.read_csv(r'/BTC-USD.csv')
df_time = df.set_index('Date')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3480 entries, 0 to 3479
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3480 non-null   object 
 1   Open       3480 non-null   float64
 2   High       3480 non-null   float64
 3   Low        3480 non-null   float64
 4   Close      3480 non-null   float64
 5   Adj Close  3480 non-null   float64
 6   Volume     3480 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 190.4+ KB


In [ ]:
df_time = df_time.reset_index()


In [ ]:
# import plotly.express as px

# fig = px.line(df, x='Date', y='Adj Close', title="BitCoin Adj closed price")
# fig.update_xaxes(
#     rangeslider_visible=True,
#     rangeselector=dict(
#         buttons=list([
#             dict(count=1, label="1m", step="month", stepmode="backward"),
#             dict(count=6, label="6m", step="month", stepmode="backward"),
#             dict(count=1, label="YTD", step="year", stepmode="todate"),
#             dict(count=1, label="1y", step="year", stepmode="backward"),
#             dict(step="all")
#         ])
#     )
# )
# fig.show()


In [ ]:
def MACD(DF,a = 6,b= 13,c = 5):
    df = DF.copy()
    df['ma_fast'] = df['Adj Close'].ewm(span = a,min_periods = a).mean()
    df['ma_slow'] = df['Adj Close'].ewm(span = b,min_periods = b).mean()
    df['macd'] = df['ma_fast'] - df['ma_slow']
    df['macd_signal'] = df['macd'].ewm(span = 9, min_periods =c).mean()
    df = df.drop(columns = ['ma_fast', 'ma_slow', 'macd'] )
    return df

In [ ]:
df= MACD(df)
df = df.set_index('Date')
# df

In [ ]:
# df[['macd_signal']].plot()

In [ ]:
# df.columns

In [ ]:
def ATR(Df, n =7):
    df = Df.copy()
    # ATR
    df['C_H - C_L'] = df['High'] - df['Low']
    df['C_H - P_C'] = df['High'] - df['Adj Close'].shift(1)
    df['C_L - P_C'] = df['Low'] - df['Adj Close'].shift(1)
    df['TR'] = df[['C_H - C_L', 'C_H - P_C', 'C_L - P_C']].max(axis = 1, skipna = False)
    df['ATR'] = df['TR'].ewm(span = n, min_periods = n).mean()
    df = df.drop(columns= ['C_H - C_L','C_H - P_C','C_L - P_C','TR'])
    return df

In [ ]:
df = ATR(df)
# df.head(15)

In [ ]:
# df[['ATR']].plot()

In [ ]:
def BB(Df, n = 7):
    df = Df.copy()
    # BB
    df['MB'] = df['Adj Close'].rolling(n).mean()
    df['UB'] = df['MB'] + 2*df['Adj Close'].rolling(n).std(ddof = 0)
    df['LB'] = df['MB'] - 2*df['Adj Close'].rolling(n).std(ddof = 0)
    df['BB_width'] = df['UB'] - df['LB']



    return df

In [ ]:
df = BB(df)
# import plotly.graph_objects as go

# # Assuming df_final is your DataFrame
# # Create a Plotly figure
# fig = go.Figure()

# # Add traces for 'Appliances' and 'App_Pred

# fig.add_trace(go.Scatter(x=df.index, y=df['Adj Close'], mode='lines', name='actual curve'))
# fig.add_trace(go.Scatter(x=df.index, y=df['MB'], mode='lines', name='MB'))
# fig.add_trace(go.Scatter(x=df.index, y=df['UB'], mode='lines', name='UB'))
# fig.add_trace(go.Scatter(x=df.index, y=df['LB'], mode='lines', name='LB'))

# # Update layout with titles and labels
# fig.update_layout(title='BOlinger band',
#                   xaxis_title='Time',
#                   yaxis_title='Value',
#                   xaxis=dict(rangeslider=dict(visible=True)))  # Add range slider

# # Show the plot
# fig.show()


In [ ]:
df = df.drop(columns = ['MB', 'UB', 'LB'])
# df

In [ ]:
import numpy as np
def RSI(Df, n = 7):
    df = Df.copy()
    df['change'] = df['Adj Close'] - df['Adj Close'].shift(1)
    df["gain"] = np.where(df['change'] >=0, df['change'], 0)
    df["loss"] = np.where(df['change'] <=0, -1*df['change'], 0)
    df['avg_gain'] = df['gain'].ewm(alpha = 1/n, min_periods = n).mean()
    df['avg_loss'] = df['loss'].ewm(alpha = 1/n, min_periods = n).mean()
    df['rs'] = df['avg_gain']/df['avg_loss']
    df['RSI'] = 100 - (100/(1 + df['rs']))
    return df['RSI']

In [ ]:
df['RSI'] = RSI(df)
# df

In [ ]:
def ADX(Df, n=7):
    df = Df.copy()
    df['upmove'] = df['High'] - df['High'].shift()
    df['downmove'] = df['Low'].shift() - df['Low']
    df['+dm'] = np.where((df['upmove'] > df['downmove']) & (df['upmove'] > 0), df['upmove'], 0)
    df['-dm'] = np.where((df['downmove'] > df['upmove']) & (df['downmove'] > 0), df['downmove'], 0)
    df['+di'] = 100 * (df['+dm'] / df['ATR']).ewm(com=n, min_periods=n).mean()
    df['-di'] = 100 * (df['-dm'] / df['ATR']).ewm(com=n, min_periods=n).mean()
    df['DX'] = (abs(df['+di'] - df['-di']) / (df['+di'] + df['-di'])) * 100
    df['ADX'] = df['DX'].ewm(span=n, min_periods=n).mean()
    return df

In [ ]:
df = ADX(df)
# df

In [ ]:

df.drop(columns= ['upmove','downmove', '+dm', '-dm', '+di', '-di', 'DX'],inplace= True)
# df.columns

In [ ]:
bitcoin_price = pd.DataFrame(df[['Adj Close', 'macd_signal', 'ATR', 'BB_width', 'RSI', 'ADX']])
# bitcoin_price.info()

In [ ]:
bitcoin_price.dropna(inplace= True)
# bitcoin_price

In [ ]:
# bitcoin_price['Adj Close'][1:6]

In [ ]:
# len(bitcoin_price['Adj Close']-window_size+horizon)

In [ ]:
labels_list = []
feature_list = []
window_size = 15
horizon = 7
feature_array = np.array(bitcoin_price)
for i in range(0, len(bitcoin_price['Adj Close'])-(window_size+horizon-1)):
    feature_list.append(np.array(feature_array[i:i+window_size]))
    labels_list.append(np.array(bitcoin_price['Adj Close'][i+window_size:i+window_size+horizon]))

# Convert the list of arrays to a numpy array
labels = np.array(labels_list)
feature = np.array(feature_list)


In [ ]:
# feature.shape

In [ ]:
# labels.shape

In [ ]:
# scaler = MinMaxScaler()
# data_scaled = scaler.fit_transform(bitcoin_price)
#
data_scaled = bitcoin_price

In [ ]:
# data_scaled.head(15)

In [ ]:
def get_labelled_window(x, horizon=1):
    return x[:, :-horizon], x[:, -horizon:]



def make_windows(x,window_size,horizon):
    #Create a window of specific window size (add the horizon to the end of the window for fuether labelling)
    window_step = np.expand_dims(np.arange(window_size+horizon), axis =0)
    # print(window_step)
    # create a 2D array of multiple window steps(minus 1 to account for 0 indexing)
    window_indexes = window_step + np.expand_dims(np.arange(len(x)- (window_size+horizon -1)),axis = 0).T
    # print(window_indexes)

    windowed_array = x[window_indexes]

    windows, labelsx = get_labelled_window(windowed_array,horizon=horizon )

    return windows, labelsx



In [ ]:
# window_size = 15
# horizon = 7
# data_scaled = data_scaled.reset_index(drop=True)
# full_windows, full_labels = make_windows(data_scaled, window_size=window_size, horizon=horizon)
# full_windows.shape,full_labels.shape

In [ ]:
def make_train_test_splits(windows, labels, test_split = 0.2):
    split_size = int(len(windows)* (1-test_split))
    train_windows = windows[:split_size]
    train_labels = labels[:split_size]
    test_windows = windows[split_size:]
    test_labels = labels[split_size:]
    return train_windows, test_windows, train_labels, test_labels

In [ ]:
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(feature,labels)
train_windows.shape, (test_windows).shape, (train_labels).shape, (test_labels).shape

((2752, 15, 6), (688, 15, 6), (2752, 7), (688, 7))

model 8 (with out using minmax_scaler)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(
    monitor="val_loss",#quantity to monitor
    min_delta=0.0001,#Minimum change in the monnitored quantity as an improvement
    patience=20,#No of epochs with no improvement
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False
)

In [ ]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Define input layer
inputs = Input(shape=(window_size, 6))

# Define LSTM layers
lstm1 = LSTM(256, activation='relu', return_sequences=True)(inputs)
lstm2 = LSTM(128, activation='relu', return_sequences=True)(lstm1)
lstm3 = LSTM(128, activation='relu')(lstm2)

# Output layer with a single unit
outputs = Dense(horizon, activation='linear')(lstm3)

# Create model
model = Model(inputs=inputs, outputs=outputs)



# Compile model
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

# Train the model
model.fit(train_windows,train_labels,
          epochs=100, verbose=1,
          batch_size=1, validation_data=(test_windows,test_labels),callbacks=callback)

Epoch 1/100
3405/3405 [==============================] - 134s 38ms/step - loss: 3357.2751 - mae: 3357.2751 - val_loss: 11363.8848 - val_mae: 11363.8848
Epoch 2/100
3405/3405 [==============================] - 125s 37ms/step - loss: 2074.4639 - mae: 2074.4639 - val_loss: 6172.1191 - val_mae: 6172.1191
Epoch 3/100
3405/3405 [==============================] - 124s 36ms/step - loss: 2136.8721 - mae: 2136.8721 - val_loss: 13301.5645 - val_mae: 13301.5645
Epoch 4/100
3405/3405 [==============================] - 126s 37ms/step - loss: 1795.5826 - mae: 1795.5826 - val_loss: 6465.0161 - val_mae: 6465.0161
Epoch 5/100
3405/3405 [==============================] - 123s 36ms/step - loss: 1522.2441 - mae: 1522.2441 - val_loss: 5414.4580 - val_mae: 5414.4580
Epoch 6/100
3405/3405 [==============================] - 124s 36ms/step - loss: 1401.8179 - mae: 1401.8179 - val_loss: 7069.0259 - val_mae: 7069.0259
Epoch 7/100
3405/3405 [==============================] - 123s 36ms/step - loss: 1297.6102 - mae:

In [ ]:
def mean_absolute_scaled_error(y_true, y_pred, seasonality=1):
    mae = np.mean(np.abs(y_true - y_pred))
    naive_forecast = y_true[:-seasonality]
    naive_mae = np.mean(np.abs(naive_forecast - y_true[seasonality:]))
    mase = mae / naive_mae
    return mase

def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
  rmse = tf.sqrt(mse)
  huber_loss = tf.keras.losses.Huber()(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true.numpy(), y_pred.numpy(),seasonality=1)

  # Account for different sized metrics (for longer horizons, reduce to single number)
  if mae.ndim > 0: # if mae isn't already a scalar, reduce it to one by aggregating tensors to mean
    mae = tf.reduce_mean(mae)
    mse = tf.reduce_mean(mse)
    rmse = tf.reduce_mean(rmse)
    huber_loss = tf.reduce_mean(huber_loss)
    mase = tf.reduce_mean(mase)

  print("For mae, mse, rmse and hubber : lower value is better.\nmase :- A scaled eror is >1 if the forecast is worse than the naive and <1 if the forecast is better that the naive.")
  print(f"""
    "mae": {mae.numpy()},
    "mse": {mse.numpy()},
    "rmse": {rmse.numpy()},
    "huber_loss": {huber_loss.numpy()},
    "mase": {mase}
    """)
  return {
        "mae": mae.numpy(),
        "mse": mse.numpy(),
        "rmse": rmse.numpy(),
        "huber_loss": huber_loss.numpy(),
        "mase": mase.numpy()
    }

In [ ]:
forecast  = model.predict(test_windows)
preds =(forecast)


In [ ]:
 test_labels.shape, preds.shape

In [ ]:
evaluate_preds(y_true= test_labels, y_pred= preds)

For mae, mse, rmse and hubber : lower value is better.
mase :- A scaled eror is >1 if the forecast is worse than the naive and <1 if the forecast is better that the naive.

    "mae": 1225.068359375,
    "mse": 3667159.75,
    "rmse": 1409.69775390625,
    "huber_loss": 1224.568359375,
    "mase": 2.1983470916748047
    


{'mae': 1225.0684,
 'mse': 3667159.8,
 'rmse': 1409.6978,
 'huber_loss': 1224.5684,
 'mase': 2.198347}

In [ ]:
plt.figure(figsize=(10, 6))  # Adjust size as needed
plt.plot(pd.to_datetime(df_time['Date'][-len(preds):]), preds, label='Predicted')
plt.plot(pd.to_datetime(df_time['Date'][-len(preds):]), tf.squeeze(test_labels), label='Actual',linestyle = " ", marker='.')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Predicted vs Actual')
plt.legend()
plt.show()

In [ ]:
x= df_time['Date'][-len(preds):])
y = (list(preds))
values = [item[0] for item in y]

z = list(tf.squeeze(full_labels).numpy())
df_onix = pd.DataFrame({'x':x, 'y':values, 'z':z})
df_onix['x']=pd.to_datetime(df_onix['x'])

df_onix.head()

In [ ]:
# import plotly.graph_objects as go

# # Assuming df_final is your DataFrame
# # Create a Plotly figure
# fig = go.Figure()

# # Add traces for 'Appliances' and 'App_Pred'
# fig.add_trace(go.Scatter(x=df_onix['x'], y=df_onix['y'], mode='lines', name='Appliances'))
# fig.add_trace(go.Scatter(x=df_onix['x'], y=df_onix['z'], mode='lines', name='App_Pred'))

# # Update layout with titles and labels
# fig.update_layout(title='Appliances vs. App_Pred',
#                   xaxis_title='Time',
#                   yaxis_title='Value',
#                   xaxis=dict(rangeslider=dict(visible=True)))  # Add range slider

# # Show the plot
# fig.show()

In [ ]:
# test_windows.shape, (test_windows[0].reshape(1,test_windows[0].shape[0], test_windows[0].shape[1])).shape
# test_windows.reshape(test_windows.shape[0], test_windows.shape[1], 1)

In [ ]:
# test_windows[0],test_labels[0]

In [ ]:
# model.predict(test_windows[0].reshape(1,test_windows[0].shape[0], test_windows[0].shape[1])),test_labels[0]

In [ ]:
model.save('model_10_multivariate_parameter_1.h5')